In [13]:
import pandas as pd
import string
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
#Get the spam data collection 
spam_dataset = pd.read_csv("/content/SpamCollection",sep ='\t',names=['response','message'])

In [4]:
spam_dataset.head()

,response,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
spam_dataset.describe()

,response,message
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


In [6]:
#view response 
spam_dataset.groupby('response').describe()

message                                                            \
           count unique                                                top   
response                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    653  Please call our customer service representativ...   

               
         freq  
response       
ham        30  
spam        4

In [7]:
#Verify length of the messages and also add it as a new column 
spam_dataset['length'] = spam_dataset['message'].apply(len)

In [8]:
spam_dataset.head()

,response,message,length
0,ham,"Go until jurong point, crazy.. Available only ...",111
1,ham,Ok lar... Joking wif u oni...,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,ham,U dun say so early hor... U c already then say...,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",61


In [9]:
#define a function to get rid of stopwords present in the messages
def msg_process(msg):
  no_punc = [char for char in msg if char not in string.punctuation]
  no_punc = ''.join(no_punc)
  return [word for word in no_punc.split() if word.lower() not in stopwords.words('english')]

In [14]:
spam_dataset['message'].head(5).apply(msg_process)

0    [Go, jurong, point, crazy, Available, bugis, n...
1                       [Ok, lar, Joking, wif, u, oni]
2    [Free, entry, 2, wkly, comp, win, FA, Cup, fin...
3        [U, dun, say, early, hor, U, c, already, say]
4    [Nah, dont, think, goes, usf, lives, around, t...
Name: message, dtype: object

In [15]:
#start text processing with vectorizer 
from sklearn.feature_extraction.text import CountVectorizer

In [16]:
#use bag of words by applying the function and fit the data into it
bow_transformer = CountVectorizer(analyzer=msg_process).fit(spam_dataset['message'])

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:5: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  """


In [17]:
#print length of bag of words stored in the vocabulary_ attribute
print(len(bow_transformer.vocabulary_))

11425


In [18]:
msg_bow  = bow_transformer.transform(spam_dataset['message'])


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:5: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  """


In [19]:
#apply tfidf transformer and fit the bag of words into it (transformed version)
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transform = TfidfTransformer().fit(msg_bow)

In [20]:
#print shape of the tfidf 
msg_tfidf = tfidf_transform.transform(msg_bow)
print(msg_tfidf.shape)

(5572, 11425)


In [21]:
#choose naive Bayes model to detect the spam and fit the tfidf data into it
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(msg_tfidf,spam_dataset['response'])

In [26]:
#check model for the predicted and expected value say for message#2 and message#5
message = spam_dataset['message'][4]
bow_message = bow_transformer.transform([message])
tfidf = tfidf_transform.transform(bow_message)

print('predicted',spam_detect_model.predict(tfidf)[0])
print('expected',spam_dataset.response[4])


('predicted', 'ham')
('expected', 'ham')
